<a href="https://colab.research.google.com/github/NgaAdrain/TEAM_Enter_Text_Here/blob/master/Data_Varification/20200727_Deep_Learning_Model_With_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<h1>**Deep Learning With CNN**</h1>

##**Module Import**


In [2]:
# Import the TensorFlow
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

##**File Read**

###<h2>학습용 데이터는 Google Drive에 train_input.csv의 형식으로 저장한다.</h2>

In [3]:
train_file = pd.read_csv('/content/drive/My Drive/train_input.csv',encoding='utf-8')
#train_file = pd.read_csv('/content/drive/My Drive/train_input_1.csv',encoding='utf-8')
#train_file = pd.read_csv('/content/drive/My Drive/train_input_external_city.csv',encoding='utf-8')
#train_file = pd.read_csv('/content/drive/My Drive/train_input_internal_city.csv',encoding='utf-8')
train_file_label = train_file.pop('Label')
#train_file.drop(['Timestamp','Winker(left)','Winker(right)','Timecheck'],axis = 'columns',inplace = True)
train_file.drop(['Timestamp','accel_Z','Winker(left)','Winker(right)','Timecheck'],axis = 'columns',inplace = True)
#print(train_file.values)
#if file broken -> sep='  ' 추가

###<h2>시뮬레이션 테스트 데이터는 test_input.csv의 형식으로 저장한다.</h2>

In [4]:
#test_file = pd.read_csv('/content/drive/My Drive/test_input.csv',encoding='utf-8')
test_file = pd.read_csv('/content/drive/My Drive/test_input_1.csv',encoding='utf-8')
#test_file = pd.read_csv('/content/drive/My Drive/test_input_1_internal.csv',encoding='utf-8')
#test_file = pd.read_csv('/content/drive/My Drive/test_input_2_external.csv',encoding='utf-8')
test_file_time = test_file.pop('Timestamp')
test_file_label = test_file.pop('Label')
test_file.drop(['accel_Z','Winker(left)','Winker(right)','Timecheck'],axis = 'columns',inplace = True)

###<h2>실제 데이터는 real_data.csv의 형식으로 저장한다.</h2>

In [6]:
#real_file = pd.read_csv('/content/drive/My Drive/real_data.csv',sep='',encoding='utf-8')
real_file = pd.read_csv('/content/drive/My Drive/real_data.csv',encoding='utf-8')
real_file_label = real_file.pop('Label')
real_file.drop('accel_Z',axis = 'columns',inplace = True)

##**DataFrame 2 Numpy Array**


###<h2>학습용 데이터 변환</h2>

In [7]:
train_array = np.delete(train_file.values,-1,0)
train_label = np.delete(train_file_label.values,-1,0)

###<h2>테스트 데이터 변환</h2>

In [8]:
test_array = np.delete(test_file.values,-1,0)
test_label = np.delete(test_file_label.values,-1,0)
test_time = np.delete(test_file_time.values,-1,0)

###<h2>실제 데이터 변환</h2>

In [9]:
real_array = np.delete(real_file.values,-1,0)
real_label = np.delete(real_file_label.values,-1,0)

###<h3>데이터 길이</h3>

In [10]:
length_test = int(len(test_array)/25)
length_train = int(len(train_array)/25)

##**데이터 정규화**

###사용 상수

###<h2>학습용 데이터 정규화</h2>

- <h3>실제 데이터의 정규화내용</h3>
1. steering wheel  -> 31.75(270,-270) ~ -32
2. brake -> 31.75(50,0) ~ -32
3. speed(velocity) -> 31.75(100,0) ~ -32
4. accel -> 31.75(70,0) ~ -32
5. accelaration_longitudinal(X) -> 31.75(0.6,-0.6) ~ -32
6. accelaration_lateral(Z) -> 31.75(0.6,-0.6) ~ -32

In [12]:
print("Process 1-1 Velocity[",end="")
for i in range(len(train_array)):
  train_array[i][VELOCITY] = round(train_array[i][VELOCITY],0)
  if(i%length_train==0):
    print("#",end="")
  if(train_file.values[i][VELOCITY]>=100):    #MAX
    train_array[i][VELOCITY] = 31.75
  elif(train_file.values[i][VELOCITY]>=50):   #MID
    train_array[i][VELOCITY] = round(((train_array[i][VELOCITY] - 50)/50*31.75),2)
  elif(train_file.values[i][VELOCITY]>0):     #MIN
    train_array[i][VELOCITY] = round(((train_array[i][VELOCITY] - 50)/50*32),2)
  else:
    train_array[i][VELOCITY] = -32
print("]\nProcess 1-2 Accel_X [",end="")
for i in range(len(train_array)):
  if(i%length_train==0):
    print("#",end="")  
  if(train_file.values[i][ACCEL_X]>=0.6):     #MAX
    train_array[i][ACCEL_X] = 31.75
  elif(train_file.values[i][ACCEL_X]>= 0):    #MID
    train_array[i][ACCEL_X] = round(((train_array[i][ACCEL_X])/0.6*31.75),2)
  elif(train_file.values[i][ACCEL_X]> -0.6):  #MIN
    train_array[i][ACCEL_X] = round(((train_array[i][ACCEL_X])/0.6*32),2)
  else:
    train_array[i][ACCEL_X] = -32
"""
print("]\nProcess 1-3 Accel_Z [",end="")
for i in range(len(train_array)):
  if(i%length_test==0):
    print("#",end="")
  if(train_file.values[i][ACCEL_Z]>=1):       #MAX
    train_array[i][ACCEL_Z] = 31.75
  elif(train_file.values[i][ACCEL_Z]>= 0):    #MID
    train_array[i][ACCEL_Z] = round(((train_array[i][ACCEL_Z])*31.75),2)
  elif(train_file.values[i][ACCEL_Z]>-1):     #MIN
    train_array[i][ACCEL_Z] = round(((train_array[i][ACCEL_Z])*32),2)
  else:
    train_array[i][ACCEL_Z] = -32
"""
print("]\nProcess 1-3 Steering[",end="")
for i in range(len(train_array)):
  if(i%length_train==0):
    print("#",end="")  
  if(train_file.values[i][STEERING]>=270):    #MAX
    train_array[i][STEERING] = 31.75
  elif(train_file.values[i][STEERING]>= 0):   #MID
    train_array[i][STEERING] = round(((train_array[i][STEERING])/270*31.75),2)
  elif(train_file.values[i][STEERING]> -270): #MIN
    train_array[i][STEERING] = round(((train_array[i][STEERING])/270*32),2)
  else:
    train_array[i][STEERING] = -32
print("]\nProcess 1-4 Accelpad[",end="")
for i in range(len(train_array)):
  if(i%length_train==0):
    print("#",end="")  
  if(train_file.values[i][ACCEL]>=70):        #MAX
    train_array[i][ACCEL] = 31.75
  elif(train_file.values[i][ACCEL]>= 35):     #MID
    train_array[i][ACCEL] = round(((train_array[i][ACCEL] - 35)/35*31.75),2)
  elif(train_file.values[i][ACCEL]> 0):       #MIN
    train_array[i][ACCEL] = round(((train_array[i][ACCEL] - 35)/35*32),2)
  else:
    train_array[i][ACCEL] = -32
print("]\nProcess 1-5 Brakepad[",end="")
for i in range(len(train_array)):
  if(i%length_train==0):
    print("#",end="")
  if(train_file.values[i][BRAKE]>=50):        #MAX
    train_array[i][BRAKE] = 31.75
  elif(train_file.values[i][BRAKE]>= 25):     #MID
    train_array[i][BRAKE] = round(((train_array[i][BRAKE] - 25)/30*31.75),2)
  elif(train_file.values[i][BRAKE]> 0):       #MIN
    train_array[i][BRAKE] = round(((train_array[i][BRAKE] - 25)/30*32),2)
  else:
    train_array[i][BRAKE] = -32
print("]\nProcess 1 Clear")

Process 1-1 Velocity[##########################]
Process 1-2 Accel_X [##########################]
Process 1-3 Steering[##########################]
Process 1-4 Accelpad[##########################]
Process 1-5 Brakepad[##########################]
Process 1 Clear


###<h2>테스트 데이터 정규화</h2>

- <h3>시뮬레이터 값의 범위</h3>
1. Velocity : 0 ~ 240 -> 31.75(100,0) ~ -32
2. Acceleration.<br>
Z -> 속도가 감소하면 양수, 속도가 증가하면 음수 -> 31.75(1,-1) ~ -32<br>
X -> 차가 왼쪽으로 이동하면 감소 오른쪽으로 이동하면 증가 -> 31.75(0.6,-0.6) ~ -32
3. Steering_wheel : -1 ~ 1 => -450 ~ 450(degree) -> 31.75(270,-270) ~ -32
4. Brake : 0 ~ 1 => 0 ~ 100% -> 31.75(60,0) ~ -32
5. Accel : 0 ~ 1 => 0 ~ 100% -> 31.75(80,0) ~ -32<br>

In [13]:
print("Process 2-1 Velocity[",end="")
for i in range(len(test_array)):
  test_array[i][VELOCITY] = round(test_array[i][VELOCITY],0)
  if(i%length_test==0):
    print("#",end="")
  if(test_file.values[i][VELOCITY]>=100):   #MAX
    test_array[i][VELOCITY] = 31.75
  elif(test_file.values[i][VELOCITY]>=50):  #MID
    test_array[i][VELOCITY] = round(((test_array[i][VELOCITY] - 50)/50*31.75),2)
  elif(test_file.values[i][VELOCITY]>0):    #MIN
    test_array[i][VELOCITY] = round(((test_array[i][VELOCITY] - 50)/50*32),2)
  else:                                     
    test_array[i][VELOCITY] = -32
print("]\nProcess 2-2 Accel_X [",end="")
for i in range(len(test_array)):
  if(i%length_test==0):
    print("#",end="")  
  if(test_file.values[i][ACCEL_X]>=0.6):    #MAX
    test_array[i][ACCEL_X] = 31.75
  elif(test_file.values[i][ACCEL_X]>= 0):   #MID
    test_array[i][ACCEL_X] = round(((test_array[i][ACCEL_X])/0.6*31.75),2)
  elif(test_file.values[i][ACCEL_X]> -0.6): #MIN
    test_array[i][ACCEL_X] = round(((test_array[i][ACCEL_X])/0.6*32),2)
  else:                                     
    test_array[i][ACCEL_X] = -32
"""
print("]\nProcess 2-3 Accel_Z [",end="")
for i in range(len(test_array)):
  if(i%length_test==0):
    print("#",end="")
  if(test_file.values[i][ACCEL_Z]>=1):      #MAX
    test_array[i][ACCEL_Z] = 31.75
  elif(test_file.values[i][ACCEL_Z]>= 0):   #MID
    test_array[i][ACCEL_Z] = round(((test_array[i][ACCEL_Z])*31.75),2)
  elif(test_file.values[i][ACCEL_Z]>-1):    #MIN
    test_array[i][ACCEL_Z] = round(((test_array[i][ACCEL_Z])*32),2)
  else:                                     
    test_array[i][ACCEL_Z] = -32
"""
print("]\nProcess 2-3 Steering[",end="")
for i in range(len(test_array)):
  if(i%length_test==0):
    print("#",end="")  
  if(test_file.values[i][STEERING]>=270):   #MAX
    test_array[i][STEERING] = 31.75
  elif(test_file.values[i][STEERING]>= 0):  #MID
    test_array[i][STEERING] = round(((test_array[i][STEERING])/270*31.75),2)
  elif(test_file.values[i][STEERING]>-270): #MIN
    test_array[i][STEERING] = round(((test_array[i][STEERING])/270*32),2)
  else:                                     
    test_array[i][STEERING] = -32
print("]\nProcess 2-4 Accelpad[",end="")
for i in range(len(test_array)):
  if(i%length_test==0):
    print("#",end="")  
  if(test_file.values[i][ACCEL]>=70):       #MAX
    test_array[i][ACCEL] = 31.75
  elif(test_file.values[i][ACCEL]>= 35):    #MID
    test_array[i][ACCEL] = round(((test_array[i][ACCEL] - 35)/35*31.75),2)
  elif(test_file.values[i][ACCEL]> 0):      #MIN
    test_array[i][ACCEL] = round(((test_array[i][ACCEL] - 35)/35*32),2)
  else:                                     
    test_array[i][ACCEL] = -32
print("]\nProcess 2-5 Brakepad[",end="")
for i in range(len(test_array)):
  if(i%length_test==0):
    print("#",end="")
  if(test_file.values[i][BRAKE]>=50):       #MAX
    test_array[i][BRAKE] = 31.75
  elif(test_file.values[i][BRAKE]>= 25):    #MID
    test_array[i][BRAKE] = round(((test_array[i][BRAKE] - 25)/25*31.75),2)
  elif(test_file.values[i][BRAKE]> 0):      #MIN
    test_array[i][BRAKE] = round(((test_array[i][BRAKE] - 25)/25*32),2)
  else:                                     
    test_array[i][BRAKE] = -32
print("]\nProcess 2 Clear")

Process 2-1 Velocity[##########################]
Process 2-2 Accel_X [##########################]
Process 2-3 Steering[##########################]
Process 2-4 Accelpad[##########################]
Process 2-5 Brakepad[##########################]
Process 2 Clear


##**데이터셋 만들기**

###<h2>30Hz->10Hz로 데이터 갯수 맞추기</h2>

In [64]:
train_10hz_array = []
train_10hz_label = []
count = 0
for temp in train_array:
  if(count%3 == 1):
    train_10hz_array.append(train_array[count,:])
    train_10hz_label.append(train_label[count])
  count = count + 1
train_10hz_array = np.array(train_10hz_array)
train_10hz_label = np.array(train_10hz_label)
print(len(train_10hz_array))
print(len(train_10hz_label))

68669
68669


In [76]:
test_10hz_array = []
test_10hz_label = []
count = 0
for temp in test_array:
  if(count%3 == 1):
    test_10hz_array.append(test_array[count,:])
    test_10hz_label.append(test_label[count])
  count = count + 1
test_10hz_array = np.array(test_10hz_array)
test_10hz_label = np.array(test_10hz_label)
print(test_10hz_array)
print(len(test_10hz_label))

[[ 13.34   2.07  -0.12 -32.   -32.  ]
 [ 13.34   5.77   0.   -32.   -32.  ]
 [ 12.7    5.85   0.12 -32.   -32.  ]
 ...
 [-32.     0.     0.12 -32.   -32.  ]
 [-32.    -0.     0.12 -32.   -32.  ]
 [-32.    -0.     0.12 -32.   -32.  ]]
6057


###<h1>사용 상수</h1>

In [14]:
WINDOW_SIZE = 150 #초당 32개의 데이터가 나옴.(30~32Hz)(window)// 120개(4초 분량)

In [67]:
WINDOW_10HZ_SIZE = 50 #초당 10개의 데이터로 줄임.

###<h2>Deep Neural Network Dataset</h2>

####<h2>학습용 데이터셋</h2>

In [15]:
#30hz
train_dataset_data = []
train_dataset_label = []
for i in range(WINDOW_SIZE, train_array.shape[0] + 1, 1):
  train_dataset_data.append(train_array[i-WINDOW_SIZE:i])
  train_dataset_label.append(train_label[i-1])
tr_data = np.array(train_dataset_data)
tr_label = np.array(train_dataset_label) 

print(tr_data.shape)
print(tr_label.shape)

(205858, 150, 5)
(205858,)


In [68]:
#10hz
train_dataset_data = []
train_dataset_label = []
for i in range(WINDOW_10HZ_SIZE, train_10hz_array.shape[0] + 1, 1):
  train_dataset_data.append(train_10hz_array[i-WINDOW_10HZ_SIZE:i])
  train_dataset_label.append(train_10hz_label[i-1])
tr_10hz_data = np.array(train_dataset_data)
tr_10hz_label = np.array(train_dataset_label)
 
print(tr_10hz_data.shape)
print(tr_10hz_label.shape)

(68620, 50, 5)
(68620,)


####<h2>테스트 데이터셋</h2>

In [73]:
#30hz
test_dataset_data = []
test_dataset_label = []
for i in range(WINDOW_SIZE, test_array.shape[0] + 1, 1):
  test_dataset_data.append(test_array[i-WINDOW_SIZE:i])
  test_dataset_label.append(test_label[i-1])

te_data = np.array(test_dataset_data)
te_label = np.array(test_dataset_label) 
print(tr_data.shape)

(205858, 150, 5)


In [81]:
#10hz
test_dataset_data = []
test_dataset_label = []
for i in range(WINDOW_10HZ_SIZE, test_10hz_array.shape[0] + 1, 1):
  test_dataset_data.append(test_10hz_array[i-WINDOW_10HZ_SIZE:i])
  test_dataset_label.append(test_10hz_label[i-1])

te_10hz_data = np.array(test_dataset_data)
te_10hz_label = np.array(test_dataset_label) 
print(te_10hz_data.shape)

(6008, 50, 5)


####<h2>실제 데이터셋</h2>

In [17]:
real_dataset_data = []
real_dataset_label = []
for i in range(WINDOW_SIZE, real_array.shape[0] + 1, 1):
  real_dataset_data.append(real_array[i-WINDOW_SIZE:i])
  real_dataset_label.append(real_label[i-1])
re_data = np.array(real_dataset_data)
re_label = np.array(real_dataset_label) 

In [79]:
real_dataset_data = []
real_dataset_label = []
for i in range(WINDOW_10HZ_SIZE, real_array.shape[0] + 1, 1):
  real_dataset_data.append(real_array[i-WINDOW_10HZ_SIZE:i])
  real_dataset_label.append(real_label[i-1])
re_10hz_data = np.array(real_dataset_data)
re_10hz_label = np.array(real_dataset_label) 
print(re_10hz_data.shape)

(53212, 50, 5)


###<h2>Convolution Neural Network Dataset</h2>

위의 np.array에 차원을 추가

In [145]:
##여기 구현
##TO DO: data의 구조를 (None,총 row수, WINDOW크기, 라벨수)로 바꿔야함. 4차원으로 변경

##**상태 분석 모델링**

###<h2>Deep Neural Network Model</h2>

In [35]:
dnn_model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(WINDOW_SIZE, 6)),
    keras.layers.Flatten(input_shape=(WINDOW_SIZE, 5)),
    #keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(64),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(128),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(16),
    #keras.layers.Dense(16),
    keras.layers.Dense(8, activation='softmax')
])
dnn_model.summary()
dnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 750)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               96128     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 136       
Total params: 114,904
Trainable params: 114,904
Non-trainable params: 0
________________________________________________

In [82]:
dnn_10hz_model = keras.Sequential([
    #keras.layers.Flatten(input_shape=(WINDOW_SIZE, 6)),
    keras.layers.Flatten(input_shape=(WINDOW_10HZ_SIZE, 5)),
    #keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(64),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(128),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(16),
    #keras.layers.Dense(16),
    keras.layers.Dense(8, activation='softmax')
])
dnn_10hz_model.summary()
dnn_10hz_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32128     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_13 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 136       
Total params: 50,904
Trainable params: 50,904
Non-trainable params: 0
__________________________________________________

###<h2>Convolution Neural Network Model</h2>

Convolution neural Network에서 합성곱 신경망의 필터를 통과할 때(Conv2D-> 2차원 필터) 데이터는 차원이 낮아지게 된다.<br>
NxN의 vector가 FxF의 필터를 통과하게되면 (N-F+1)x(N-F+1)로 줄어들게 된다.<br>
즉 지금 여기의 예시에서는 120(윈도우 크기) x 5를 3x3이 통과하게 되면 118 x 3으로 줄어든다.<br>
이를 MaxPool2D의 2x2필터를 통과하게 되면 117x2로 줄어들게 되며 다시한번 Conv2D의 2x2 필터를 통과하게 되면 116x1로 줄어들어서 더이상 Convolution필터를 통과할 수 없게 된다.<br>
이 때문에 많은 필터 layer를 추가하지 못하므로 이를 해결하기위해 padding 기술을 사용한다.<br>
padding기술은 각 edge의 값을 복사하여 각 방향으로 한 칸 늘려서 크기를 유지하는 기술이다.<br>
Conv2D에서는 padding을 사용하지 않는 valid, padding을 사용하는 same의 두가지를 지원한다. 이를 사용하면 많은 layer를 넣을 수 있다. 이를 이용하여 해결하였다.


In [126]:
cnn_model = keras.Sequential([
    keras.layers.Conv2D(64,(3,3),activation='relu',padding='same',input_shape = (WINDOW_SIZE,5,1)),#120개(WINDOW_SIZE의 3x3 필터) #??????????????????? => 왜 INPUT SHAPE가 120X18X1로 들어가지???????
    #keras.layers.Conv2D(120,(3,3),activation='relu', input_shape = (WINDOW_SIZE,5,1))
    keras.layers.MaxPool2D((2,2),padding='same'), #2x2 size의 Max Pooling 2D Matrix
    #keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'), # 전 단계 필터 수의 2배인 240개의 3x3 matrix filters
    keras.layers.MaxPool2D((2,2),padding='same'), #2x2 size의 Max Pooling 2D Matrix
    keras.layers.Conv2D(64,(3,3),padding='same'), # 전 단계 필터 수의 1/2배인 120개의 3x3 matrix filters
    #keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu') ,
    keras.layers.Dense(8, activation='softmax')   #stop 넣는 경우
    #keras.layers.Dense(7, activition='softmax')  #stop 빼는 경우
])
cnn_model.summary()
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_84 (Conv2D)           (None, 150, 5, 64)        640       
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 75, 3, 64)         0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 75, 3, 128)        73856     
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 38, 2, 128)        0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 38, 2, 64)         73792     
_________________________________________________________________
flatten_34 (Flatten)         (None, 4864)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 64)              

In [ ]:
"""
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(120, (3, 3), activation='relu', input_shape=(WINDOW_SIZE,18,1)))#??????????????????? => 왜 INPUT SHAPE가 120X18X1로 들어가지???????
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(240, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(120, (3, 3), activation='relu'))
"""

##**모델 학습**

###DNN MODEL 학습

In [36]:
dnn_model.fit(tr_data,tr_label,epochs=30,batch_size=32)

Epoch 1/25
6434/6434 [==============================] - 16s 2ms/step - loss: 0.7243 - accuracy: 0.8037
Epoch 2/25
6434/6434 [==============================] - 18s 3ms/step - loss: 0.4800 - accuracy: 0.8484
Epoch 3/25
6434/6434 [==============================] - 15s 2ms/step - loss: 0.4202 - accuracy: 0.8662
Epoch 4/25
6434/6434 [==============================] - 15s 2ms/step - loss: 0.3839 - accuracy: 0.8770
Epoch 5/25
6434/6434 [==============================] - 16s 3ms/step - loss: 0.3621 - accuracy: 0.8835
Epoch 6/25
6434/6434 [==============================] - 17s 3ms/step - loss: 0.3452 - accuracy: 0.8885
Epoch 7/25
6434/6434 [==============================] - 16s 3ms/step - loss: 0.3317 - accuracy: 0.8923
Epoch 8/25
6434/6434 [==============================] - 16s 3ms/step - loss: 0.3231 - accuracy: 0.8949
Epoch 9/25
6434/6434 [==============================] - 16s 3ms/step - loss: 0.3147 - accuracy: 0.8968
Epoch 10/25
6434/6434 [==============================] - 16s 2ms/step - l

In [83]:
dnn_10hz_model.fit(tr_10hz_data,tr_10hz_label,epochs=30,batch_size=32)

Epoch 1/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.9483 - accuracy: 0.7717
Epoch 2/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.5669 - accuracy: 0.8258
Epoch 3/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.5008 - accuracy: 0.8415
Epoch 4/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.4655 - accuracy: 0.8527
Epoch 5/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.4285 - accuracy: 0.8614
Epoch 6/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.4081 - accuracy: 0.8674
Epoch 7/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.3859 - accuracy: 0.8758
Epoch 8/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.3710 - accuracy: 0.8801
Epoch 9/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.3578 - accuracy: 0.8840
Epoch 10/30
2145/2145 [==============================] - 4s 2ms/step - loss: 0.345

###CNN MODEL 학습

In [ ]:
cnn_model.fit(tr_c_data,tr_c_label,epochs=30,batch_size=32)

##**모델 평가**

###<h2>Deep Neural Network Model</h2>

####테스트 데이터 평가

In [37]:
dnn_model.evaluate(te_data, te_label,verbose=2)

564/564 - 1s - loss: 0.9974 - accuracy: 0.7942


[0.9973881244659424, 0.7942073941230774]

In [85]:
dnn_10hz_model.evaluate(te_10hz_data, te_10hz_label,verbose=2)

188/188 - 0s - loss: 1.0036 - accuracy: 0.7939


[1.003562331199646, 0.7939414381980896]

####실제 데이터 평가

In [38]:
dnn_model.evaluate(redata, re_label,verbose=2)

1660/1660 - 2s - loss: 1.5128 - accuracy: 0.7755


[1.5128397941589355, 0.77545565366745]

In [86]:
dnn_10hz_model.evaluate(re_10hz_data, re_10hz_label,verbose=2)

1663/1663 - 2s - loss: 1.3938 - accuracy: 0.7672


[1.3937963247299194, 0.7672141790390015]

###<h2>Convolution Neural Network Model</h2>

####테스트 데이터 평가

In [ ]:
cnn_model.evaluate(te_c_data, te_label,verbose=2)

####실제 데이터 평가

In [ ]:
cnn_model.evaluate(re_c_data, te_label,verbose=2)

##<h2>**모델 예측**</h2>

In [87]:
predictions = dnn_10hz_model.predict(re_10hz_data)

In [88]:
txt = 'Pred_Direction,Real_Direction\n'
count = 0
for temp in re_data:
  if(np.argmax(predictions[count]) == 0):
  #  print('straight')
    txt = txt + ',straight'
  elif(np.argmax(predictions[count]) == 1):
  #  print('left')
    txt = txt + ',curve_left'
  elif(np.argmax(predictions[count]) == 2):
    txt = txt + ',curve_right'
  elif(np.argmax(predictions[count]) == 3):
    txt = txt + ',left' 
  elif(np.argmax(predictions[count]) == 4):
    txt = txt + ',right'
  elif(np.argmax(predictions[count]) == 5):
    txt = txt + ',change_left'
  elif(np.argmax(predictions[count]) == 6):
    txt = txt + ',change_right'
  else:
    txt = txt + ',stop'
  if(real_file_label[count] == 0):
    txt = txt + ',straight\n'
  elif(real_file_label[count] == 1):
    txt = txt + ',curve_left\n'
  elif(real_file_label[count] == 2):
    txt = txt + ',curve_right\n'
  elif(real_file_label[count] == 3):
    txt = txt + ',left\n'
  elif(real_file_label[count] == 4):
    txt = txt + ',right\n' 
  elif(real_file_label[count] == 5):
    txt = txt + ',change_left\n'
  elif(real_file_label[count] == 6):
    txt = txt + ',change_right\n'
  else:
    txt = txt + ',stop\n'
  count = count + 1

In [89]:
f = open("/content/drive/My Drive/real_pred_output.csv",'w')
f.write(txt)
f.close()